## Import libaries 

In [1]:
import pandas as pd
import networkx as nx
%pylab inline
import numpy as np
from random import *
from numpy import *
from random import *
import numpy.linalg as la
from scipy.linalg import eig
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, f1_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt

/Users/maisha/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/maisha/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Populating the interactive namespace from numpy and matplotlib


## Function for import data and merge the datasets 

In [2]:
def import_data(filename):
    """
    This function is used how to import a dataset from a file.
    """
    data=pd.read_csv(filename)
    return data
def drop_rows(df, n):
    """
    This is the function for remove rows from a dataframe. 
    """
    return df.iloc[:-n] if n < len(df) else pd.DataFrame(columns=df.columns)


In [3]:
filename='Anthropometry_and_diabetes.csv'
data= pd.read_csv(filename)

In [34]:
## Total Number of diabetic people
diabetic_count = data[(data['dibaHbA1C'] ==1)].shape[0]

In [35]:
## Total Number of hypertension people
hypertension_count = data[(data['htnbin'] ==1 )].shape[0]


In [43]:
## Total Number of ability to work people 
ability_to_work_count = data[(data['binary_ability'] ==1 )].shape[0]


##  Train and Test Data 

In [6]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=16)

## Function for finding participant characteristics (mean, median, standard deviation, maximum, minimum)

In [17]:
X=data[['age2018','Weight_kg','Height_cm','BMI','WholeBodyFatPerc','TrunkPerc','meangripboth','generate waistc','generate waisthp','generate waistht','generate_hipc']]
def basic_property(X):
    '''
    This function is used to compute the characteristics such as mean,median, standard deviation, maximum, and minimum
    of the participant from the dataset.
    '''
  
    mean=X.mean() #mean
    median=X.median() #median
    stndd=X.std()#standard deviation
    maximum=X.max() # maximum
    minimum=X.min() #minimum 
    return mean, median, stndd,maximum,minimum

## Decision tree classification  

In [ ]:
X=data[['Weight_kg','Height_cm','BMI','WholeBodyFatPerc','TrunkPerc','meangripboth','generate waistc','generate waisthp','generate waistht','generate_hipc']]

## Test Data y is changed according to the target

In [ ]:
y=data['dibaHbA1C'] # For Diabetes
y=data['htnbin'] # For Hypertension
y=data['binary_ability']  # For ability to work

In [15]:
def decisiontreeclassification(X_train, y_train):
    '''
    This function is used to find the feature importance of the variable by decision tree classifier.
    '''
    clf = DecisionTreeClassifier(random_state=42, class_weight="balanced")
    output = cross_validate(clf, X_train, y_train, cv=3, return_estimator=True)

    all_importances = []
    for estimator in output['estimator']:
        all_importances.append(estimator.feature_importances_)

    mean_importance = np.mean(all_importances, axis=0)
    std_importance = np.std(all_importances, axis=0)

    feature_importances = pd.DataFrame({
        'importance_mean': mean_importance,
        'importance_std': std_importance
    }, index=X_train.columns).sort_values('importance_mean', ascending=False)

    return feature_importances


## Logistic regression models 

In [11]:
# Model 1 for Diabetes
X1=data[['age2018','generate waistc','generate waisthp','Weight_kg','Height_cm','BMI']]
# Model 2 for Diabetes
X2=data[['age2018','generate waistc','generate waisthp','Weight_kg','Height_cm']]
# Model 3 for Diabetes
X3=data[['age2018','generate waistc','generate waisthp','Weight_kg']]
# Model 4 for Diabetes
X4=data[['age2018','generate waistc','generate waisthp']]
# Model 5 for Diabetes
X5=data[['age2018','generate waistc']]

In [12]:
# Model 1 for Hypertension
X1=data[['age2018','WholeBodyFatPerc','generate waistht','generate waisthp','generate_hipc','generate waistc']]
# Model 2 for Hypertension
X2=data[['age2018','WholeBodyFatPerc','generate waistht','generate waisthp','generate_hipc']]
# Model 3 for Hypertension
X3=data[['age2018','WholeBodyFatPerc','generate waistht','generate waisthp']]
# Model 4 for Hypertension
X4=data[['age2018','WholeBodyFatPerc','generate waistht']]
# Model 5 for Hypertension
X5=data[['age2018','WholeBodyFatPerc']]

In [13]:
# Model 1 for ability to work
X1=data[['age2018','BMI','meangripboth','generate_hipc','TrunkPerc','Weight_kg']]
# Model 2 for ability to work
X2=data[['age2018','BMI','meangripboth','generate_hipc','TrunkPerc']]
# Model 3 for ability to work
X2=data[['age2018','BMI','meangripboth','generate_hipc']]
# Model 4 for ability to work
X4=data[['age2018','BMI','meangripboth']]
# Model 5 for ability to work
X4=data[['age2018','BMI']]

In [14]:
def logisticregresssionmodel(X_train, y_train):
    '''
    This function is used to find the coefficient value for each feature by the logistic refression model. X_train is changed 
    based on the model X1,X2,X3,X4 and X5 and y_train is changed according to the target.
    '''
    mod_log = OrderedModel(y_train,X_train,distr='logit')
    res_log = mod_log.fit(method='bfgs', disp=False)
    return res_log.summary()

## Percentile figure plot  for Diabetes

In [ ]:
data_Fd=data[data['dibaHbA1C']==1]
data_Fnd =data[data['dibaHbA1C']==0]
percentiles = np.linspace(0,1,1000)
r1 = data_Fd['generate waistc'].quantile(percentiles)
r2 = data_Fnd['generate waistc'].quantile(percentiles)
plt.plot(percentiles,r1)
plt.plot(percentiles,r2)
legend(['Diabetic Female','Non-diabetic Female'],fontsize=18)
plt.xlabel('Percentiles',fontsize=18)
plt.ylabel('Freuency',fontsize=18)

## Percentile plot for Hypertension

In [ ]:
data_Fd =data[data['htnbin']==1]
data_Fnd =data[data['htnbin']==0]
percentiles = np.linspace(0,1,1000)
r1 = data_Fd['WholeBodyFatPerc'].quantile(percentiles)
r2 = data_Fnd['WholeBodyFatPerc'].quantile(percentiles)
plt.plot(percentiles,r1)
plt.plot(percentiles,r2)
legend(['Hypertension Female','Non-hypertension Female'],fontsize=18,loc='upper right')
plt.xlabel('Percentiles',fontsize=18)
plt.ylabel('Freuency',fontsize=18)

## Percentile Plot for ability to work

In [ ]:
data_Fd =data[data['binary_ability']==1]
data_Fnd =data[data['binary_ability']==0]
percentiles = np.linspace(0,1,1000)
r1 = data_Fd['meangripboth'].quantile(percentiles)
r2 = data_Fnd['meangripboth'].quantile(percentiles)
plt.plot(percentiles,r1)
plt.plot(percentiles,r2)
legend(['Female who are able to do work','Female who are not able to do work'],fontsize=14,loc='upper right')
plt.xlabel('Percentiles',fontsize=18)
plt.ylabel('Freuency',fontsize=18)

## ROC curve plot code

In [ ]:
logreg = LogisticRegression(max_iter=1000)
model_score=logreg.fit(X_train, y_train)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
plt.ylabel('True Positive Rate',fontsize=18)
plt.xlabel('False Positive Rate',fontsize=18)
plt.legend()
plt.show()

##  Waist circumference sensitivity and specificity for diabetes across potential screening cutoff values

In [ ]:
subset_df = data[(data['generate waistc'] >= 70) & (data['generate waistc'] <= 100)]
# Calculate sensitivity and specificity
sensitivity_values = []
specificity_values = []
thresholds = []

for threshold in range(70, 101):
    threshold_df = subset_df.copy()
    threshold_df['predicted_diabetes'] = (threshold_df['generate waistc'] >= threshold).astype(int)
    tn, fp, fn, tp = metrics.confusion_matrix(threshold_df['dibaHbA1C'], threshold_df['predicted_diabetes']).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    sensitivity_values.append(sensitivity)
    specificity_values.append(specificity)
    thresholds.append(threshold)


plt.figure(figsize=(10, 6))
plt.plot(thresholds, sensitivity_values, label='Sensitivity')
plt.plot(thresholds, specificity_values, label='Specificity')
plt.xlabel('Waist circumference cutoff value',fontsize=18)
plt.ylabel('Sensitivity/Specifity',fontsize=18)
plt.legend()
plt.grid(True)
plt.show()